In [123]:
%matplotlib inline 

import numpy as np
import pandas as pd

from skfeature.function.information_theoretical_based import CMIM
from sklearn.cross_validation import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import xgboost as xgb

from collections import Counter
from datetime import datetime

%run ../predict_voting_outcomes/data_preparation.py
%run ../predict_voting_outcomes/feature_preprocessing.py

np.random.seed(2016) # set seed

In [11]:
train = pd.read_csv('../data/train2016.csv')
test  = pd.read_csv('../data/test2016.csv')
sub   = pd.read_csv('../data/sampleSubmission2016.csv')

In [12]:
# concat training and test data
data = pd.concat((train, test))

In [13]:
# feature engineering
def engineer_features(data):
    data = missing_value_features(data)
    data['Age'] = data.YOB.map(create_age)
    data['Age_Mapping'] = data.Age.map(create_age_mapping)
    
    return data

data = engineer_features(data)
data = remove_outliers(data)

features = data.columns.drop(['USER_ID'])

data = fill_missing_values(data[features])
data = encode_cat_features(data[features])

In [14]:
# consider only this feature age mapping and see if it is of any importance
mask = (data.Party.notnull())

X = data.loc[mask, features.drop('Party')]
y = (data[mask].Party == 'Democrat').astype(int)

Xtest = data.loc[~mask, features.drop('Party')]

** Only include discrete variables for now **

In [23]:
X = X.select_dtypes(include=['int64'])
Xtest = Xtest.select_dtypes(include=['int64'])

** Split into training and test splits **

In [24]:
Xtr, Xte, ytr, yte = train_test_split(X, y, stratify=y, test_size=0.2, random_state=12386)

** Feature Selection **

In [129]:
feature_indices = CMIM.cmim(Xtr.values, ytr.values, n_selected_features=10)
selected_features = X.columns[feature_indices]

In [130]:
# est = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=1231)
est = xgb.XGBClassifier(seed=1231)
skf = StratifiedKFold(ytr, n_folds=5, random_state=123)

cv_scores = cross_val_score(est, Xtr[selected_features], ytr, scoring='accuracy', cv=skf, n_jobs=-1)

In [131]:
print('Mean cv-score: %f and std: %f'%(np.mean(cv_scores), np.std(cv_scores)))

Mean cv-score: 0.616328 and std: 0.019989


In [132]:
est.fit(Xtr[selected_features], ytr)
yhat = est.predict(Xte[selected_features])
print('Accuracy on held out set: %f'%(accuracy_score(yte, yhat)))

Accuracy on held out set: 0.636866


** Bagging Different Classifiers **

In [103]:
class BaggingClassifier:
    
    @staticmethod
    def majority_vote(preds):
        """
        Given an array of predictions from various classifiers
        return single array with ensemble of predictions based on
        simple majority voting

        Input: list of list [[y1, y2, y3, ..], [y1, y2, y3, ...], ..] 
        Output: final prediction [y1, y2, y3, ..]
        """
        length = [len(pred) for pred in preds]

        if len(set(length)) != 1:
            raise ValueError('Predictions must be of the same length')

        pred_matrix = np.matrix(preds)
        ensemble_preds = []

        for i in range(len(preds[0])):
            pred_column = np.array(pred_matrix[:, i]).ravel()
            common_pred = Counter(pred_column)
            most_common = common_pred.most_common()[0][0]

            ensemble_preds.append(most_common)

        return ensemble_preds
    
    def __init__(self, estimators, voting='hard'):
        """
        Estimators will be a list of tuples of (n_selected_features, estimator)
        """
        
        self.estimators = estimators
        self.fitted_models = []
        
    def fit(self, X, y=None):
        
        for n_selected_features, model in self.estimators:
            feature_indices = CMIM.cmim(X.values, y.values, n_selected_features=n_selected_features)
            selected_features = X.columns[feature_indices]
            model.fit(X[selected_features], y)    
            self.fitted_models.append((selected_features, model))
            
        return self
    
    def predict(self, X):
        preds = []
        for selected_features, model in self.fitted_models:
            yhat = model.predict(X[selected_features])
            preds.append(yhat)
        
        final_preds = self.majority_vote(preds)
        return final_preds

In [97]:
pipe1 = (5, RandomForestClassifier(n_estimators=75, max_depth=7, random_state=123))
pipe2 = (5, xgb.XGBClassifier(seed=12313))
pipe3 = (5, xgb.XGBClassifier(seed=2313))
pipe4 = (5, xgb.XGBClassifier(seed=211131))

estimators = [pipe1, pipe2, pipe3, 4]

In [136]:
bag = BaggingClassifier(estimators)
bag.fit(Xtr, ytr)

In [137]:
yhat = bag.predict(Xte)
print('Accuracy on the held out set:%f '%(accuracy_score(yte, yhat)))

Accuracy on the held out set:0.623963 


** Full Training **

In [138]:
bag = BaggingClassifier(estimators)
bag.fit(X, y)
predictions = bag.predict(Xtest)

In [ ]:
est.fit(X[selected_features], y)
predictions = est.predict(Xtest[selected_features])

In [139]:
prediction_labels = map(lambda x: 'Democrat' if x == 1 else 'Republican', predictions)
sub['Predictions'] = list(prediction_labels)
timestamp = str(datetime.now()).replace(' ', '_')
sub.to_csv('../submissions/bag_rf_xgb%s.csv'%(timestamp), index=False)